# Gather MITRE ATT&CK technique data for analysis
### Input: line-separated text file with list of MITRE ATT&CK technique IDs (e.g. T1078)
### Output: Excel file with three tabs: Datasource requirements, Platform requirements and Tactic 


### Import required libraries. Define filepath.

In [ ]:
from attackcti import attack_client
import json
import numpy
import pandas as pd
import numpy as np
import xlsxwriter

inputFile = 'techniques' #Enter the full path and file name of the text file containing the MITRE techniques you would like analyzed
outputFile = 'output.xlsx' ##Enter the full path and file name of the Excel (xlsx) file that you would like as output of this script

### Load the MITRE ATT&CK Enterprise framework

In [ ]:
lift = attack_client()
%time all_enterprise = lift.get_all_enterprise()

### Read list of techniques from a file

In [ ]:
techniques = []
techniques = [line.rstrip() for line in open(inputFile)]

In [ ]:
print(techniques)

### Generate a list of Datasource requirements for each technique

In [ ]:
dataSourceAnalysis = []
for technique in techniques:
    for t in all_enterprise['techniques']:
        if t['external_references'][0]['external_id'] == technique:
                for source in t['x_mitre_data_sources']:
                    techniqueListItem = {} 
                    techniqueListItem['ID'] = t['external_references'][0]['external_id']
                    techniqueListItem['Name'] = t['name']
                    techniqueListItem['DataSource'] = source
                    dataSourceAnalysis.append(techniqueListItem)

### Load the list of datasources into a dataframe

In [ ]:
dataSources = pd.DataFrame(dataSourceAnalysis)

### Verify that dataframe is populated by reviewing counts of datasources

In [ ]:
pd.DataFrame(dataSourceAnalysis)

In [ ]:
dataSources['DataSource'].value_counts()


### Generate a list of tactics for each technique

In [ ]:
tacticAnalysis = []
for technique in techniques:
    for t in all_enterprise['techniques']:
        if t['external_references'][0]['external_id'] == technique:
                for tactic in t['kill_chain_phases']:
                    techniqueListItem = {} 
                    techniqueListItem['ID'] = t['external_references'][0]['external_id']
                    techniqueListItem['Name'] = t['name']
                    techniqueListItem['Tactic'] = tactic['phase_name']
                    tacticAnalysis.append(techniqueListItem)

### Load the tactic list into a dataframe

In [ ]:
tactics = pd.DataFrame(tacticAnalysis)

### Confirm the dataframe is populated by reviewing per-tactic counts

In [ ]:
tactics['Tactic'].value_counts()


### Generate a list of platform requirements for each technique

In [ ]:
platformAnalysis = []
for technique in techniques:
    for t in all_enterprise['techniques']:
        if t['external_references'][0]['external_id'] == technique:
                for platform in t['x_mitre_platforms']:
                    techniqueListItem = {} 
                    techniqueListItem['ID'] = t['external_references'][0]['external_id']
                    techniqueListItem['Name'] = t['name']
                    techniqueListItem['Platform'] = platform
                    platformAnalysis.append(techniqueListItem)

### Load the platforms list into a dataframe

In [ ]:
platforms = pd.DataFrame(platformAnalysis)

### Confirm the dataframe is populated by reviewing the per-platform count

In [ ]:
platforms['Platform'].value_counts()

### Write out the Datasource, Tactic and Platforms dataframes into an Excel workbook

In [ ]:
with pd.ExcelWriter(outputFile) as writer:
    dataSources.to_excel(writer, sheet_name='Data Sources')
    tactics.to_excel(writer, sheet_name='Tactics')
    platforms.to_excel(writer, sheet_name='Platforms')